# **24. HuggingFace 커스텀 프로젝트 만들기 [프로젝트]**

## **24-1. 프로젝트 : 커스텀 프로젝트 직접 만들기**

실습 코드에서 수행해 본 내용을 토대로, 이번에는 한국어 데이터셋에 도전해보겠습니다.

앞서 본 GLUE benchmark의 한국어 버전 [KLUE benchmark](https://klue-benchmark.com/)를 들어보신 적 있나요?

GLUE와 마찬가지로 한국어 자연어처리에 대한 이해도를 높이기 위해 만들어진 데이터셋 benchmark입니다. 총 8가지의 데이터셋이 있습니다. 다만 이번 시간에 진행할 프로젝트는 KLUE의 dataset을 활용하는 것이 아닌, model(klue/ber-base)를 활용하여 NSMC(Naver Sentiment Movie Corpus) task를 도전해보겠습니다.

모델과 데이터에 관한 정보는 링크를 참조해주세요.

- [KLUE/Bert-base](https://huggingface.co/klue/bert-base)
- [NSMC](https://github.com/e9t/nsmc)

<br>
준비가 되셨다면 아래와 같은 순서로 진행해주세요.

### **라이브러리 버전을 확인해 봅니다.**

---

사용할 라이브러리 버전을 둘러봅시다.

In [1]:
# import tensorflow
# import numpy
# import transformers
# import datasets

# print(tensorflow.__version__)
# print(numpy.__version__)
# print(transformers.__version__)
# print(datasets.__version__)

### **STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성**

---

- 데이터셋은 깃허브에서 다운받거나, [Huggingface datasets](https://huggingface.co/datasets)에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [2]:
# 사용할 모듈 설정
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datasets
from datasets import load_dataset
import os
import transformers
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [3]:
# nsmc 데이터를 'load_dataset'으로 불러온 다음 Dataset 구조 출력
nsmc = load_dataset('nsmc')
print(nsmc)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


`train`, `test`를 각각 Key로 가지는 Dataset을 확인하였다.

In [4]:
# 데이터 크기 확인과 5개 데이터 확인 (train)
print(f"train length: {len(nsmc['train']['id'])}")
print("===========")
# 5개의 데이터만 확인해보기
cols = [key for key in nsmc['train']]
cols[:5]

train length: 150000


[{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0},
 {'id': '3819312',
  'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  'label': 1},
 {'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0},
 {'id': '9045019', 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'label': 0},
 {'id': '6483659',
  'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  'label': 1}]

### **STEP 2. klue/bert-base model 및 tokenizer 불러오기**

---

In [5]:
# 모델을 변경하기 편하게 변수로 설정
model_name = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### **STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기**

---

In [6]:
# 문장 전처리 함수 선언
def preprocess_text(text):
    MAX_LEN = 60
    
    return tokenizer(
        text, 
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN)

In [7]:
# train 데이터셋을 가지고 8:2로 분리 (train, val)
train_dataset, val_dataset = train_test_split(
    nsmc['train'], 
    test_size = 0.2, 
    random_state = 42
)

In [8]:
# test 데이터셋
test_dataset = nsmc['test']

In [9]:
# 문장 전처리
train_dataset['document'] = [preprocess_text(sentence) for sentence in train_dataset['document']]
val_dataset['document'] = [preprocess_text(sentence) for sentence in val_dataset['document']]

In [10]:
# 훈련 인자 설정
training_args = TrainingArguments(
    # 사전 학습된 모델의 batch size
    per_device_train_batch_size=512,
    per_device_eval_batch_size=16,
    output_dir=os.getenv('HOME')+'/aiffel/transformers',
    num_train_epochs=1,
    evaluation_strategy="epoch",
    learning_rate=2e-5
)

In [11]:
# 훈련 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # 검증 데이터셋을 설정합니다.
)

print("슝~")

슝~


In [12]:
# 훈련 실행
trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 1
  Instantaneous batch size per device = 512
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 1


KeyError: 2

In [ ]:
val_dataset['document'][2]
# val_dataset['label'][2]

In [ ]:
# 테스트 데이터로 모델 평가
results = trainer.evaluate(test_dataset)

In [ ]:
# 평가 결과 출력
print("Results:", results)

### **STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기**

---

- 데이터 전처리, `TrainingArguments` 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

### **STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교**
- 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.
<br><br>
    - [Data Collator](https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator)
    - [Trainer.TrainingArguments 의 `group_by_length`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)
<br><br>
- STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.

---

## 개인 회고

1. 이전에 배웠던 것들이 엄청 다양해서 머릿속에서 뒤죽박죽 된 적이 많았다.
    - BERT 때도 그랬고 ㅠ
2. 지금 수준에선 좋아하거나 친숙한 형태의 구성을 모듈화 시켜놨다가 자꾸 써보는 식으로 전체 flow를 접근해야할거 같다.
    - DLthon에서 내가 담당한 부분과 가까워서 따라가다가 더 꼬여버린 부분도 있었다(...)
3. 어찌저찌 복구하고는 있는데 Train()에서 왜 index 에러가 나는걸까..
    - 17:49 기준 전처리부터 이상하게 접근해버린걸 이해는 했는데 수업시간내에 완성은 안될거 같고..
4. 초기에 metric접근부터 뭔가 이상하게 단계를 밟아나간거 같다 -_-..

#### **루브릭**
아래의 기준을 바탕으로 프로젝트를 평가합니다.

|평가문항|상세기준|
|--|--|
|1. 모델과 데이터를 정상적으로 불러오고, 작동하는 것을 확인하였다.|klue/bert-base를 NSMC 데이터셋으로 fine-tuning 하여, 모델이 정상적으로 작동하는 것을 확인하였다.|
|2. Preprocessing을 개선하고, fine-tuning을 통해 모델의 성능을 개선시켰다.|Validation accuracy를 90% 이상으로 개선하였다.|
|3. 모델 학습에 Bucketing을 성공적으로 적용하고, 그 결과를 비교분석하였다.|Bucketing task을 수행하여 fine-tuning 시 연산 속도와 모델 성능 간의 trade-off 관계가 발생하는지 여부를 확인하고, 분석한 결과를 제시하였다.|